In [1]:
import sys
import os
import os.path as osp
import argparse
import numpy as np
import cv2
import torch
import json
from torch.nn.parallel.data_parallel import DataParallel
import torch.backends.cudnn as cudnn

sys.path.insert(0, osp.join('..', 'main'))
sys.path.insert(0, osp.join('..', 'data'))
sys.path.insert(0, osp.join('..', 'common'))
from config import cfg
from model import get_model
from utils.preprocessing import load_video, augmentation, generate_patch_image, process_pose
from utils.vis import vis_keypoints
from SNUEngine import SNUModel
from skeleton_function import skeletons, transform_joint, TSM_edges, angle_list, center_of_mass_1
import time
import mediapipe as mp
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [2]:
with open(osp.join('exercise_dict_1.json')) as f:
    exer_dict = json.load(f)
exer_num = len(exer_dict)
joint_num = 24

In [70]:
TIMER

-1

In [10]:
img_path = './demo_input/Squat_1.mp4'
TIMER = int(5)

cap = cv2.VideoCapture(0)
image_list = []
sk_list = []
sk_list_3d = []
sk_com_3d = []
a_list = []

prev = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print('END')
        break

    fps = cap.get(cv2.CAP_PROP_FPS)
    # Recolor Image to RGB
    h, w = frame.shape[:2]
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Make Detection

    results = pose.process(image)

    #Recolor back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if TIMER >= 0:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, str(TIMER), (50, 150), font, 6, (0, 0, 255), 5, cv2.LINE_AA)
        cv2.imshow('Mediapipe Feed', image)
        cur = time.time()
        if cur-prev >= 1:
            prev = cur
            TIMER = TIMER-1
        
    else:
        try:

            image_list.append(frame)
            lm = results.pose_landmarks
            lm_2d = lm.landmark
            lm_3d = results.pose_world_landmarks.landmark

            joint_2d = transform_joint(lm_2d, 2)
            joint_3d = transform_joint(lm_3d, 3)

            joint_2d[:,0] = joint_2d[:,0]*w
            joint_2d[:,1] = joint_2d[:,1]*h
            joint_2d = joint_2d.astype(int)


            sk_list.append(joint_2d)
            sk_list_3d.append(joint_3d)

            kp_list, com, com_3d = center_of_mass_1(joint_2d, joint_3d, frame)
            sk_com_3d.append(com_3d)
            com_x = int(com[0])
            com_y = int(com[1])
            cv2.circle(image, (com_x, com_y), 7, (0, 0, 255), -1)

            ######
            a_list.append(angle_list(joint_3d))
            ######

            for x, y in joint_2d:
                cv2.circle(image, (x,y), 6, (255, 0, 255), -1)

            for a,b in TSM_edges:
                cv2.line(image, joint_2d[a], joint_2d[b], (0, 255, 0), 3, 4)

        except:
            pass

        cv2.imshow('Mediapipe Feed', image)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [11]:
image_list = image_list[:-40]
sk_list = sk_list[:-40]

In [12]:
np.array(image_list).shape #145

(182, 480, 640, 3)

In [34]:
image_list_1 = [image_list[15*i] for i in range(len(image_list)//15)]
sk_list_1 = [sk_list[15*i] for i in range(len(sk_list)//15)]

In [32]:
np.array(sk_list_1).shape

(8, 24, 2)

In [16]:
np.array(sk_list).shape

(172, 24, 2)

In [47]:
cudnn.benchmark = True
cfg.set_args('0', 'exer', -1)
model_path = '../output/model_dump/exer/snapshot_4.pth.tar'
model = SNUModel(model_path, exer_num, joint_num)
# exer_out = model.run_video(image_list, sk_list)
exer_out = model.run_video(image_list_1, sk_list_1)
exer_idx = np.argmax(exer_out)
for k in exer_dict.keys():
    if exer_dict[k]['exercise_idx'] == exer_idx:
        exer_name = k
        break
print('Exercise type: ' + exer_name)

>>> Using GPU: 0
Stage: exer
Load checkpoint from ../output/model_dump/exer/snapshot_4.pth.tar
Exercise type: 덤벨 벤트오버 로우


In [37]:
exer_out.argmax()

12

In [48]:
exer_out*100

array([2.3478317e+00, 6.2128011e-02, 7.9478965e+00, 7.0054069e-02,
       2.7207780e-01, 4.4003688e-03, 2.2900817e-01, 2.1297988e-02,
       1.1526944e-01, 1.6241362e-02, 7.4037824e-05, 8.6727643e-01,
       7.9681229e+01, 5.4108379e-03, 5.3556901e-01, 1.4250073e+00,
       2.4538979e-02, 3.2098076e-01, 1.4471242e-03, 7.7987246e-02,
       8.5539293e-01, 2.2472218e-01, 1.5636663e+00, 4.5380691e-01,
       2.3374765e-01, 5.9221651e-02, 1.1481967e+00, 1.3869802e+00,
       4.1549047e-03, 3.2054363e-03, 3.7322799e-03, 3.8341773e-03,
       3.7581208e-03, 4.4987141e-03, 3.0326785e-03, 3.7723370e-03,
       3.4297474e-03, 3.5108435e-03, 3.6497666e-03, 3.0256212e-03,
       4.9353288e-03], dtype=float32)

In [38]:
exer_dict.keys()

dict_keys(['스탠딩 사이드 크런치', '스탠딩 니업', '버피 테스트', '스텝 포워드 다이나믹 런지', '스텝 백워드 다이나믹 런지', '사이드 런지', '크로스 런지', '굿모닝', '프런트 레이즈', '업라이트로우', '바벨 스티프 데드리프트', '바벨 로우', '덤벨 벤트오버 로우', '바벨 데드리프트', '바벨 스쿼트', '바벨 런지', '오버 헤드 프레스', '사이드 레터럴 레이즈', '바벨 컬 ', '덤벨 컬', '딥스', '풀업', '행잉 레그 레이즈', '랫풀 다운', '페이스 풀', '케이블 크런치', '케이블 푸시 다운', '로잉머신', '라잉 레그 레이즈', '크런치', '바이시클 크런치', '시저크로스', '힙쓰러스트', '플랭크', '푸시업', '니푸쉬업', 'Y - Exercise', '덤벨 체스트 플라이', '덤벨 인클라인 체스트 플라이', '덤벨 풀 오버', '라잉 트라이셉스 익스텐션'])

In [49]:
np.argpartition(exer_out, -5)

array([32, 40, 39,  3,  4,  5,  6,  7,  8,  9, 10, 38, 37, 13, 14, 36, 16,
       17, 18, 19,  1, 21, 35, 23, 24, 25, 34, 33, 28, 29, 30, 31, 20, 26,
       27, 11, 15, 12, 22,  2,  0], dtype=int64)

In [58]:
ind = np.argpartition(exer_out, -5)[-5:]

In [69]:
n = 5
for i in ind[np.argsort(exer_out[ind])]:
    for index, k in enumerate(exer_dict.keys()):
        if i == index:
            
            print('%d : ' %(n), k)
            n -= 1
      

5 :  바벨 런지
4 :  행잉 레그 레이즈
3 :  스탠딩 사이드 크런치
2 :  버피 테스트
1 :  덤벨 벤트오버 로우


### 상대적 평가를 하는 방법

In [13]:
arr = np.load('PyQt/data/list_set_1.npz')['image_list']

In [14]:
for frame in arr:
    img = frame.astype(np.uint8).copy() # np.float32 -> np.uint8
    cv2.imshow('show', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(0.03)
cv2.destroyAllWindows()